In [1]:
import pandas as pd
import numpy as np
import gensim
import re
from pathlib import Path
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from nltk.sentiment import SentimentIntensityAnalyzer
from matplotlib import pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tag import pos_tag

import nltk
nltk.download('vader_lexicon')

# %pip install Ipython
from IPython.display import display, Markdown


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/amittaijoel/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


You don't have to manually process the files, I created a HuggingFace Dataset.
You can easily import it and convert it to pandas as below.

_Note: you may have to run `pip install datasets` first._

In [2]:
# %pip install datasets
import datasets

# for more details see https://huggingface.co/datasets/siavava/ai-tech-articles
dataset = datasets.load_dataset("siavava/ai-tech-articles")
df = dataset["train"].to_pandas()  # there's only the 'train' split because HF requires it.
# display(df.head(5))

# split df by year
years = df["year"].unique()
dfs = {}
for year in years:
    dfs[year] = df[df["year"] == year]
    
    # limit to 5 (just for testing)
    dfs[year] = dfs[year].head(5)
    
for year in sorted(years, reverse=True):
    display(Markdown(f"### Year: {year}"))
    display(Markdown(f"Number of articles: {len(dfs[year])}"))
    display(dfs[year].head(3))
    display(Markdown("---"))
    


### Year: 2023

Number of articles: 5

,id,year,title,url,text
0,0,2023,"""MIT Technology Review""","""https://www.technologyreview.com""","""Featured Topics Newsletters Events Podcasts F..."
1,1,2023,"""WIRED - The Latest in Technology, Science, Cu...","""https://www.wired.com""","""Open Navigation Menu To revisit this article,..."
3,3,2023,"""TechCrunch | Startup and Technology News""","""https://www.techcrunch.com""","""WeWork reportedly on the verge of filing for ..."


---

### Year: 2022

...

---

### Year: 2001

Number of articles: 3

,id,year,title,url,text
1122,1155,2001,"""Chromosome territories, nuclear architecture ...","""https://doi.org/10.1038%2F35066075""","""Thank you for visiting nature.com. You are us..."
1142,1183,2001,"""Department Of Computer Science | Makerere Uni...","""https://cs.mak.ac.ug/curriculum""","""Makerere University Department of Computer Sc..."
1143,1184,2001,"""Department Of Computer Science | Makerere Uni...","""https://cs.mak.ac.ug/curriculum/undergraduate...","""Makerere University Department of Computer Sc..."


---

The data is separated by year. I will be starting from 2000 and I removed the unknown sub-folder within the data folder Each of the folders will be converted to dataframes that correspond to a specific year with rows that represent each of the files.

Each of the files in the folders will be first go through a content check (if the document is too short (empty) or too long (over 1000 lines) it will be discarded).

If the file passes the length check it will then be shortened to remove the header and have its contents added to the dataframe.

This dataframe will then be added to a dictionary that maps years to the corresponding dataframe. 


> I think this is good filtering. You can easily do this in the dfs generated above.

In [3]:
# def file_process(filename):
    
#     file = open(filename, "r")
#     num_lines = len(file.readlines())
    
#     if num_lines < 15 or num_lines > 400:
#         file.close()
#         return None
    
#     else:
#         file.seek(0) 
#         content = file.readlines()[5:]

#     text = " ".join(content)
#     file.close()
    
#     return text


In [4]:
#print(file_process("../data/categorized/2023/440.txt"))


In [5]:
# def create_dataframe(foldername):

#     folder = Path(foldername)
#     df = pd.DataFrame(columns=['ID', 'Text'])
#     index = 0

#     for filename in sorted(folder.iterdir()):
#         if filename.is_file() == True:
#             text = file_process(filename)
#             if text != None:
#                 id = re.search(r'(\d+)\.txt$', str(filename))
#                 id = int(id.group(1))
#                 df.loc[index] = [str(id),text]
#                 index = index + 1
    
#     return df


In [6]:
#display(create_dataframe('../data/categorized/2013'))


In [7]:
# def create_dictionary(data_folder):
#     year_dict = {}
#     folder = Path(data_folder)
#     begin = False

#     for year_folder in sorted(folder.iterdir()):
        
#         if year_folder.is_dir() == True:
#             year_key = re.search(r'categorized/(\d+)', str(year_folder))
#             year_key = int(year_key.group(1))
            
#             if year_key == 2000:
#                 begin = True
        
#             if begin == True:
#                 year_df = create_dataframe(year_folder)

#                 if year_df.empty == False:
#                     year_dict[year_key] = year_df
    
#     return year_dict


In [8]:
# year_dictionary = create_dictionary('../data/categorized')

year_dictionary = dfs


Now that we have the year dictionary we can begin the process of sentiment analysis.

The sentiment analysis model will return a sentiment number for each file. This number will then be averaged across all the files to get an average sentiment for the year.

From there we can do cosine similarity and plot sentiment for each year.

In [9]:
def add_sentiment(dataframe):

    sentiment_model = SentimentIntensityAnalyzer()
    neg_list = []
    neu_list = []
    pos_list = []
    comp_list = []


    for text in dataframe['text']:
        score_dict = sentiment_model.polarity_scores(text)
        neg_list.append(score_dict['neg'])
        neu_list.append(score_dict['neu'])
        pos_list.append(score_dict['pos'])
        comp_list.append(score_dict['compound'])
    
    dataframe['Negative'] = neg_list
    dataframe['Neutral'] =  neu_list
    dataframe['Positive'] = pos_list
    dataframe['Compound'] = comp_list


In [10]:
# text = ["CS89 is an amazing class!"]

# df = pd.DataFrame({'Text': text})

# add_sentiment(df)
# display(df)


In [11]:
for df in year_dictionary.values():
    add_sentiment(df)


In [12]:
def get_sentiment(df, sentiment_type):
    return np.mean(df[sentiment_type])


In [13]:
def get_sentiment_data(year_dict):
    
    sentiment_types = ['Negative', 'Neutral', 'Positive', 'Compound']
    sentiment_df = pd.DataFrame(columns=['Year', 'Negative', 'Neutral', 'Positive', 'Compound'])
    index = 0

    for year, df in year_dict.items():
        data_list = [int(year)]

        for s_type in sentiment_types:
            data_list.append(get_sentiment(df,s_type))
            
        sentiment_df.loc[index] = data_list
        index = index + 1

    return sentiment_df


In [14]:
sentiment_data_df = get_sentiment_data(year_dictionary)
display(sentiment_data_df)


,Year,Negative,Neutral,Positive,Compound
0,2023.0,0.053800,0.79380,0.152600,0.997080
1,2019.0,0.029600,0.84660,0.123600,0.997800
2,2022.0,0.029000,0.85320,0.117800,0.999260
3,2016.0,0.028400,0.83480,0.136800,0.998400
4,2020.0,0.062400,0.83760,0.100000,0.606860
5,2021.0,0.053800,0.84860,0.097400,0.606740
6,2012.0,0.031000,0.87020,0.098800,0.597420
7,2017.0,0.021800,0.86120,0.116800,0.998660
8,2018.0,0.017200,0.83800,0.144800,0.998280
9,2008.0,0.022800,0.86560,0.111600,0.998080
